# [문제 1] Fashion MNIST 데이터 정규화를 위한 Mean과 Std. 값 찾기

In [14]:
import torch
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms
from wandb.util import download_file_into_memory

In [25]:
data_path = "."
f_mnist_train = datasets.FashionMNIST(data_path, train=True, download=True, transform=transforms.ToTensor())
f_mnist_train, f_mnist_validation = random_split(f_mnist_train, [55_000, 5_000])

images = torch.stack([img for img, _ in f_mnist_train], dim=0)

mean = images.mean()
std = images.std()

print(f"Mean: {mean.item():.4f}")
print(f"Std: {std.item():.4f}")

Mean: 0.2860
Std: 0.3530


# [문제 2] Fashion MNIST 데이터에 대하여 CNN 학습시키기

In [1]:
!rm -rf link_dl
!git clone https://github.com/wjm0423/link_dl.git

Cloning into 'link_dl'...
remote: Enumerating objects: 2856, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 2856 (delta 171), reused 302 (delta 124), pack-reused 2505 (from 2)
Receiving objects: 100% (2856/2856), 60.57 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (1940/1940), done.


In [2]:
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 20.2 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 44 kB 99.2 MB/s  eta 0:00:01
     |████████████████████████████████| 404 kB 79.5 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 151.8 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.4.0 requires dask==2023.3.2, but you have dask 2022.5.1 which is incompatible.
dask-cudf 23.4.0 requires distributed==2023.3.2.1, but you have distributed 2022.5.1 which is incompatible.
dask-cuda 23.4.0 requires dask==2023.3.2, but you have dask 2022.5.1 which is incompatible.
dask-cuda 23.4.0 requires distributed==2023.3.2.1, but you have distributed 2022.5.1 which is incompatibl

In [3]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/work/.netrc
wandb: Currently logged in as: wjm0423 (wjm0423-korea-university-of-technology-and-education) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
!pip install torchinfo

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [8]:
import torch
from torch import nn, optim
from datetime import datetime
import os
from pathlib import Path
from torch.optim import lr_scheduler

try:
  BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)  # BASE_PATH: /Users/yhhan/git/link_dl
except NameError:
  BASE_PATH = str(Path.cwd() / 'link_dl')
import sys
sys.path.append(BASE_PATH)
os.listdir(BASE_PATH)

try:
  CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
except NameError:
  CURRENT_FILE_PATH = str(Path.cwd())
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")
os.makedirs(CHECKPOINT_FILE_PATH, exist_ok=True)
if not os.path.isdir(CHECKPOINT_FILE_PATH):
  os.makedirs(CHECKPOINT_FILE_PATH)

import sys
sys.path.append(BASE_PATH)

from _01_code._09_fcn_best_practice.c_trainer import ClassificationTrainer
from _03_homeworks.homework_3.a_fashion_mnist_data import get_fashion_mnist_data
from _01_code._16_modern_cnns.a_arg_parser import get_parser


def get_resnet_model(num_classes=10):
    class ResnetBlock(nn.Module):

      def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()

        # ------------------------------------
        # ResNet 블록: H(x) = x + F(x)의 F(x)
        # ------------------------------------
        # ResNet 블록의 첫 번째 Convolution Layer
        # 입력 피처 맵(in_channels)을 받아 out_channels개의 3 × 3 필터를 학습하여 출력 피처 맵을 생성
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False
        )
        # 배치 정규화: 각 피처에 대해 배치 단위로 평균과 분산을 계산해 정규화
        self.bn1 = nn.BatchNorm2d(out_channels)
        # 활성화 함수-ReLU: 입력값 x가 0 이하면 0, 0 초과면 x 반환
        self.relu = nn.ReLU(inplace=True)

        # ResNet 블록의 첫 번째 Convolution Layer 이후 Convolution Layer
        # 블록 내부에서 필터링만 수행
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        # H(x) = x + F(x)에서 x와 F(x)의 크기가 다를 때 x(=identity)에 적용하는 다운샘플링
        # stride가 1이 아닐 때 크기 차이가 발생하기 때문에 stride != 1 or self.in_channels != out_channels로 조건을 준다.
        self.downsample = downsample

      # ResNet 블록의 순전파 순서
      def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # stride != 1 or self.in_channels != out_channels이면 identity에 다운샘플링을 적용하여 out과 크기를 맞춤.
        if self.downsample is not None:
          identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

    # ------------------------------------
    # ResNet-18
    # ------------------------------------
    class ResNet18(nn.Module):
      def __init__(self):
        super().__init__()

        # 처음 stem 부분 → Conv 사용
        # 입력: Fashion-MNIST 이미지 → 1 × 28 × 28
        # Fashion-MNIST 이미지는 흑백 이미지이기 때문에 RGB값이 없어 채널 값은 1
        # B × 1 × 28 × 28 --> B × 64 × {(28 - 3 + 2) / 1 + 1} × {(28 - 3 + 2) / 1 + 1} = B × 64 × 28 × 28
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=1, bias=False),
            nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1),
            nn.ReLU(inplace=True)
        )

        self.in_channels = 64

        # ResNet stages (2 blocks × 4 layers): 각 layer는 ResNetBlock을 2개씩 쌓은 것.
        # B × 64 × 28 × 28 --> B × 64 × {(28 - 3 + 2) / 1 + 1} × {(28 - 3 + 2) / 1 + 1} = B × 64 × 28 × 28
        self.layer1 = self._make_layer(out_channels=64, blocks=2, stride=1)
        # B × 64 × 28 ×28 --> B × 128 × {(28 - 3 + 2) / 2 + 1} × {(28 - 3 + 2) / 2 + 1} = B × 128 × 14 × 14
        self.layer2 = self._make_layer(out_channels=128, blocks=2, stride=2)
        # B × 128 × 28 ×28 --> B × 256 × {(14 - 3 + 2) / 2 + 1} × {(14 - 3 + 2) / 2 + 1} = B × 256 × 7 × 7
        self.layer3 = self._make_layer(out_channels=256, blocks=2, stride=2)
        # B × 256 × 7 ×7 --> B × 512 × {(7 - 3 + 2) / 2 + 1} × {(7 - 3 + 2) / 2 + 1} = B × 512 × 4 × 4
        self.layer4 = self._make_layer(out_channels=512, blocks=2, stride=2)

        # B × 512 × 4 × 4 --> B × 512 × 1 × 1
        # 각 채널에 대해 모든 4 × 4 공간 차원의 평균값을 계산하여 1 × 1 크기의 텐서로 만듦. 즉, 각 샘플을 512개의 특징 벡터로 요약.
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.4) # p: dropout probability
        # B × 512 --> B × 10
        # 512개의 특징 벡터를 가지고 판단하여 샘플이 10개(num_classes)의 클래스별로 분류될 확률을 계산
        self.fc = nn.Linear(512, num_classes)

      def _make_layer(self, out_channels, blocks, stride):
        # ResNet 블록들을 쌓아 하나의 layer를 생성
        downsample = None

        # 필요한 경우 identity 다운샘플링
        # downsample 로직 → Conv/BatchNorm 활용
        if stride != 1 or self.in_channels != out_channels:
          downsample = nn.Sequential(
              nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
              nn.BatchNorm2d(out_channels)
        )

        # 첫 번째 ResNet 블록을 만들어 layers에 추가
        layers = []
        layers.append(ResnetBlock(self.in_channels, out_channels, stride=stride, downsample=downsample))

        self.in_channels = out_channels

        # 나머지 ResNet 블록을 만들어 layers에 추가하여 반환
        for _ in range(1, blocks):
          layers.append(ResnetBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

      def forward(self, x):
        """
        Pass the input through the net.
        """
        x = self.stem(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        # B × 512 × 1 × 1 --> B × 512
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    # ResNet18 인스턴스를 만들어 반환
    my_model = ResNet18()

    return my_model


def main(args):
  # 프로젝트 이름이나 체크포인트에 사용하기 위해 현재 시간을 타임스탬프로 생성
  run_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  # 학습 설정
  config = {
      'epochs': args.epochs,
      'batch_size': args.batch_size,
      'validation_intervals': args.validation_intervals,
      'learning_rate': args.learning_rate,
      'early_stop_patience': args.early_stop_patience,
      'early_stop_delta': args.early_stop_delta,
      'weight_decay': args.weight_decay
  }

  # Wandb에 프로젝트 로깅하여 출력
  project_name = "cnn_fashion_mnist"
  name = "resnet_{0}".format(run_time_str)
  wandb.init(
      mode="online" if args.wandb else "disabled",
      project=project_name,
      notes="fashion mnist experiment with resnet",
      tags=["resnet", "fashion_mnist"],
      name=name,
      config=config
  )
  print(args)
  print(wandb.config)

  # CUDA 가능 여부에 따라 GPU나 CPU 선택
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print(f"Training on device {device}.")

  # 학습용, 검증용 DataLoader 및 Data Augmentation을 적용한 이미지 transform을 호출
  train_data_loader, validation_data_loader, fashion_mnist_transforms = get_fashion_mnist_data()
  model = get_resnet_model(num_classes=10)
  model.to(device)

  # ResNet18 모델 생성 후 GPU/CPU 디바이스로 이동
  from torchinfo import summary
  summary(model=model,
          input_size=(1, 1, 28, 28),
          col_names=["kernel_size", "input_size", "output_size", "num_params", "mult_adds"]
  )

  # 옵티마이저로 SGD를 사용
  optimizer = optim.SGD(
      model.parameters(),
      lr=wandb.config.learning_rate,
      momentum=0.9,
      weight_decay=config['weight_decay']
  )

  # 학습률 스케줄러
  scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

  # Fashion-MNIST 데이터셋 분류 훈련용 객체 생성.
  # train_loop()로 훈련, 검증, early stopping, 체크포인트 저장, wandb 로깅 등을 수행
  classification_trainer = ClassificationTrainer(
      project_name, model, optimizer, train_data_loader, validation_data_loader, fashion_mnist_transforms,
      run_time_str, wandb, device, CHECKPOINT_FILE_PATH, scheduler=scheduler
  )
  classification_trainer.train_loop()

  # wandb 세션 종료
  wandb.finish()

if __name__ == "__main__":
  # parser = get_parser()
  # args = parser.parse_args()
  # python _01_code/_09_modern_cnns/_02_googlenet/a_cifar10_train_googlenet.py --wandb -v 10
  from types import SimpleNamespace

  args = SimpleNamespace(
      wandb=True,
      batch_size=512,
      epochs=200,
      learning_rate=0.01,
      validation_intervals=10,
      early_stop_patience=7,
      early_stop_delta=1e-05,
      weight_decay=0.0005
  )

  main(args)

Epoch,▁█
Training accuracy (%),▁█
Training loss,█▁
Training speed (epochs/sec.),▁█
Validation accuracy (%),▁█
Validation loss,█▁
Epoch,10
Training accuracy (%),97.64727
Training loss,0.07063
Training speed (epochs/sec.),0.08475
Validation accuracy (%),91.82


namespace(wandb=True, batch_size=512, epochs=200, learning_rate=0.01, validation_intervals=10, early_stop_patience=7, early_stop_delta=1e-05, weight_decay=0.0005)
{'epochs': 200, 'batch_size': 512, 'validation_intervals': 10, 'learning_rate': 0.01, 'early_stop_patience': 7, 'early_stop_delta': 1e-05, 'weight_decay': 0.0005}
Training on device cuda:0.
Num Train Samples:  55000
Num Validation Samples:  5000
Sample Data Shape:  torch.Size([1, 28, 28])
Sample Data Target:  1
Number of Data Loading Workers: 3
[Epoch   1] T_loss: 0.57706, T_accuracy: 79.2036 | V_loss: 0.31476, V_accuracy: 88.5600 | Early stopping is stated! | T_time: 00:00:12, T_speed: 0.083
[Epoch  10] T_loss: 0.02572, T_accuracy: 99.1382 | V_loss: 0.33584, V_accuracy: 91.8800 | Early stopping counter: 1 out of 7 | T_time: 00:01:58, T_speed: 0.085
[Epoch  20] T_loss: 0.00034, T_accuracy: 100.0000 | V_loss: 0.29797, V_accuracy: 93.5400 | V_loss decreased (0.31476 --> 0.29797). Saving model... | T_time: 00:03:56, T_speed: 0.0

Epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇█
Training accuracy (%),▁██████████████
Training loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training speed (epochs/sec.),▁▇▇█▇▇█████████
Validation accuracy (%),▁▅█████████████
Validation loss,▅█▃▂▁▂▁▁▁▁▁▁▁▁▁
Epoch,140
Training accuracy (%),100
Training loss,0.00024
Training speed (epochs/sec.),0.08495
Validation accuracy (%),93.68


# [문제 3] 학습 완료된 모델로 테스트 데이터 Accuracy	확인하기
# [문제 4] 샘플 테스트 데이터 분류 예측 결과 확인하기


In [ ]:
import numpy as np
import torch
import os
import random

from matplotlib import pyplot as plt
from pathlib import Path


from torch import nn, optim
from datetime import datetime

try:
    BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)  # BASE_PATH: /Users/yhhan/git/link_dl
except NameError:
    BASE_PATH = str(Path.cwd() / 'link_dl')
try:
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_FILE_PATH = str(Path.cwd())
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")
os.makedirs(CHECKPOINT_FILE_PATH, exist_ok=True)
if not os.path.isdir(CHECKPOINT_FILE_PATH):
  os.makedirs(CHECKPOINT_FILE_PATH)

import sys
sys.path.append(BASE_PATH)

from _01_code._09_fcn_best_practice.d_tester import ClassificationTester
from _03_homeworks.homework_3.a_fashion_mnist_data import get_fashion_mnist_test_data

import torchvision

USE_PYTORCH_MODEL = False

def get_resnet_model(num_classes=10):
    class ResnetBlock(nn.Module):

      def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()

        # ------------------------------------
        # ResNet 블록: H(x) = x + F(x)의 F(x)
        # ------------------------------------
        # ResNet 블록의 첫 번째 Convolution Layer
        # 입력 피처 맵(in_channels)을 받아 out_channels개의 3 × 3 필터를 학습하여 출력 피처 맵을 생성
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False
        )
        # 배치 정규화: 각 피처에 대해 배치 단위로 평균과 분산을 계산해 정규화
        self.bn1 = nn.BatchNorm2d(out_channels)
        # 활성화 함수-ReLU: 입력값 x가 0 이하면 0, 0 초과면 x 반환
        self.relu = nn.ReLU(inplace=True)

        # ResNet 블록의 첫 번째 Convolution Layer 이후 Convolution Layer
        # 블록 내부에서 필터링만 수행
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        # H(x) = x + F(x)에서 x와 F(x)의 크기가 다를 때 x(=identity)에 적용하는 다운샘플링
        # stride가 1이 아닐 때 크기 차이가 발생하기 때문에 stride != 1 or self.in_channels != out_channels로 조건을 준다.
        self.downsample = downsample

      # ResNet 블록의 순전파 순서
      def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # stride != 1 or self.in_channels != out_channels이면 identity에 다운샘플링을 적용하여 out과 크기를 맞춤.
        if self.downsample is not None:
          identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

    # ------------------------------------
    # ResNet-18
    # ------------------------------------
    class ResNet18(nn.Module):
      def __init__(self):
        super().__init__()

        # 처음 stem 부분 → Conv 사용
        # 입력: Fashion-MNIST 이미지 → 1 × 28 × 28
        # Fashion-MNIST 이미지는 흑백 이미지이기 때문에 RGB값이 없어 채널 값은 1
        # B × 1 × 28 × 28 --> B × 64 × {(28 - 3 + 2) / 1 + 1} × {(28 - 3 + 2) / 1 + 1} = B × 64 × 28 × 28
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=1, bias=False),
            nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1),
            nn.ReLU(inplace=True)
        )

        self.in_channels = 64

        # ResNet stages (2 blocks × 4 layers): 각 layer는 ResNetBlock을 2개씩 쌓은 것.
        # B × 64 × 28 × 28 --> B × 64 × {(28 - 3 + 2) / 1 + 1} × {(28 - 3 + 2) / 1 + 1} = B × 64 × 28 × 28
        self.layer1 = self._make_layer(out_channels=64, blocks=2, stride=1)
        # B × 64 × 28 ×28 --> B × 128 × {(28 - 3 + 2) / 2 + 1} × {(28 - 3 + 2) / 2 + 1} = B × 128 × 14 × 14
        self.layer2 = self._make_layer(out_channels=128, blocks=2, stride=2)
        # B × 128 × 28 ×28 --> B × 256 × {(14 - 3 + 2) / 2 + 1} × {(14 - 3 + 2) / 2 + 1} = B × 256 × 7 × 7
        self.layer3 = self._make_layer(out_channels=256, blocks=2, stride=2)
        # B × 256 × 7 ×7 --> B × 512 × {(7 - 3 + 2) / 2 + 1} × {(7 - 3 + 2) / 2 + 1} = B × 512 × 4 × 4
        self.layer4 = self._make_layer(out_channels=512, blocks=2, stride=2)

        # B × 512 × 4 × 4 --> B × 512 × 1 × 1
        # 각 채널에 대해 모든 4 × 4 공간 차원의 평균값을 계산하여 1 × 1 크기의 텐서로 만듦. 즉, 각 샘플을 512개의 특징 벡터로 요약.
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.4) # p: dropout probability
        # B × 512 --> B × 10
        # 512개의 특징 벡터를 가지고 판단하여 샘플이 10개(num_classes)의 클래스별로 분류될 확률을 계산
        self.fc = nn.Linear(512, num_classes)

      def _make_layer(self, out_channels, blocks, stride):
        # ResNet 블록들을 쌓아 하나의 layer를 생성
        downsample = None

        # 필요한 경우 identity 다운샘플링
        # downsample 로직 → Conv/BatchNorm 활용
        if stride != 1 or self.in_channels != out_channels:
          downsample = nn.Sequential(
              nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
              nn.BatchNorm2d(out_channels)
        )

        # 첫 번째 ResNet 블록을 만들어 layers에 추가
        layers = []
        layers.append(ResnetBlock(self.in_channels, out_channels, stride=stride, downsample=downsample))

        self.in_channels = out_channels

        # 나머지 ResNet 블록을 만들어 layers에 추가하여 반환
        for _ in range(1, blocks):
          layers.append(ResnetBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

      def forward(self, x):
        """
        Pass the input through the net.
        """
        x = self.stem(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        # B × 512 × 1 × 1 --> B × 512
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    # ResNet18 인스턴스를 만들어 반환
    my_model = ResNet18()

    return my_model

def main():
    f_mnist_test_images, test_data_loader, f_mnist_transforms = get_fashion_mnist_test_data()
    
    # 테스트 모델 로드
    test_model = torchvision.models.resnet18(num_classes=10) if USE_PYTORCH_MODEL else get_resnet_model(num_classes=10)
    
    # 분류 테스터 객체 생성 및 전체 정확도 평가
    project_name = "cnn_fashion_mnist"
    classification_tester = ClassificationTester(
        project_name, test_model, test_data_loader, f_mnist_transforms, CHECKPOINT_FILE_PATH
    )
    # 훈련 완료된 모델로 테스트 데이터의 정확도 확인
    classification_tester.test()
    
    # -----------------------------------------------------
    # 샘플 테스트 데이터 10개 샘플의 분류 예측 결과 확인
    # -----------------------------------------------------
    num_test_samples = len(f_mnist_test_images)
    sample_indices = random.sample(range(num_test_samples), 10)
    misclassified_found = False
    
    # Fashion MNIST 클래스 레이블 매핑
    labels_map = {
        0: "T-shirt/top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat", 
        5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle boot"
    }

    print("\n" + "="*50)
    print("                 [문제 4] 10개 샘플 예측 결과")
    print("="*50)

    for i, idx in enumerate(sample_indices):
        img_tensor, label_int = f_mnist_test_images[idx]
        
        # 모델 예측
        predicted_int = classification_tester.test_single(img_tensor)
        
        # 결과 비교
        is_correct = (predicted_int == label_int)
        
        print(f"\n--- 샘플 {i+1} (Index: {idx}) ---")
        print(f"정답 레이블 (ID): {label_int} ({labels_map[label_int]})")
        print(f"예측 결과 (ID): {predicted_int} ({labels_map[predicted_int]})")
        print(f"일치 여부: {'O (정답)' if is_correct else 'X (오분류)'}")
        
        # 오분류된 샘플의 이미지 출력 및 해석 준비
        if not is_correct:
            misclassified_found = True
            
            # 이미지 출력
            plt.figure(figsize=(2, 2))
            plt.imshow(img_tensor.squeeze().numpy(), cmap='gray')
            plt.title(f"Label: {labels_map[label_int]}, Pred: {labels_map[predicted_int]}")
            plt.show()

if __name__ == "__main__":
    main()

/home/work/link_dl


100% 26421880/26421880 [00:03<00:00, 6758032.32it/s] 


Extracting /home/work/link_dl/_00_data/j_fashion_mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/work/link_dl/_00_data/j_fashion_mnist/FashionMNIST/raw



100% 29515/29515 [00:00<00:00, 103970.95it/s]


Extracting /home/work/link_dl/_00_data/j_fashion_mnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/work/link_dl/_00_data/j_fashion_mnist/FashionMNIST/raw



100% 4422102/4422102 [00:02<00:00, 1751795.57it/s]


Extracting /home/work/link_dl/_00_data/j_fashion_mnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/work/link_dl/_00_data/j_fashion_mnist/FashionMNIST/raw



100% 5148/5148 [00:00<00:00, 6896287.76it/s]


Extracting /home/work/link_dl/_00_data/j_fashion_mnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/work/link_dl/_00_data/j_fashion_mnist/FashionMNIST/raw

Num Test Samples:  10000
Sample Shape:  torch.Size([1, 28, 28])
MODEL FILE: /home/work/checkpoints/cnn_fashion_mnist_checkpoint_latest.pt


# 숙제 후기

시험해봐야 할 하이퍼 파라미터 조합이 많은데 Backend.ai, 구글 코랩 모두 GPU로 돌려도 훈련을 한 번 시행할 때마다 최소 30분은 걸려서 시간을 매우 많이 잡아먹었습니다.

또한 Data Augmentation은 과제에 나와있는대로 f_mnist_transforms 객체에 넣어서 구현하는 게 불가능해서 과제 수행 중간에 많은 시행착오를 겪어야 했습니다. 다음 과제 때는 실행 시간을 줄이는 디폴트 하이퍼 파라미터 값이 주어지면 좋겠습니다.

심지어 마지막 테스트 데이터 실행 때는 Backend.ai의 VRAM 부족 오류로 하지도 못해 매우 안타깝게 생각합니다.